In [ ]:
# !pip install anvil-uplink

In [ ]:
import anvil.server

In [ ]:
anvil.server.connect("server_A54UXZNMAOF44RJNVYRNG6UE-QIJV2LK4TV7ORECE")

In [ ]:
# !pip install git+https://github.com/huggingface/transformers

# !pip install qwen_vl_utils


In [ ]:
import torch

In [ ]:
# Initialize an empty list to store the conversation history
conversation_history = []

images

In [ ]:
from PIL import Image
import io

# Define a function to receive the image
@anvil.server.callable
def process_uploaded_image(image_file):
    # Convert the Anvil image file to a PIL Image
    image_bytes = io.BytesIO(image_file.get_bytes())
    image = Image.open(image_bytes)

    # Process the image as needed (e.g., save, display, analyze, etc.)
    image.save('/content/test.jpg')  # Example: Display the image

    # Return a response (optional)
    return "Image uploaded and processed successfully!"


In [ ]:
from PIL import Image
img=Image.open('/content/test.jpg')
@anvil.server.callable
def prompt_text(query,img=img):
    from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
    from qwen_vl_utils import process_vision_info
    import torch
    torch.cuda.empty_cache()
    model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct", torch_dtype="auto", device_map="auto")
    processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct-GPTQ-Int4")


    # Append the new user input to the conversation history
    conversation_history.append(
        {
            "role": "user",
            "content": [
                {"type": "image", "image": img},
                {"type": "text", "text": query}

            ],
        }
    )

    # Prepare the messages with the full conversation history
    messages = conversation_history[:]

    # Process the messages
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to('cuda')

    # Inference: Generation of the output
    generated_ids = model.generate(**inputs, max_new_tokens=128)
    generated_ids_trimmed = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )

    # Append the model's response to the conversation history
    conversation_history.append(
        {
            "role": "system",
            "content": {"type": "text", "text": output_text[0]},
        }
    )
    ans=output_text[0]
    del generated_ids, generated_ids_trimmed, output_text
    del image_inputs, video_inputs, inputs, text, messages
    del model, processor
    return ans

In [ ]:
torch.cuda.empty_cache()